In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
reviews_dict = pd.read_csv(r"/kaggle/input/airbnb-listings-reviews/Airbnb Data/Reviews_data_dictionary.csv")

In [ ]:
reviews_dict

In [ ]:
reviews = pd.read_csv(r"/kaggle/input/airbnb-listings-reviews/Airbnb Data/Reviews.csv")

In [ ]:
reviews

In [ ]:
listings_dict = reviews = pd.read_csv(r"/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings_data_dictionary.csv")

In [ ]:
listings_dict

In [ ]:
listings = pd.read_csv(r"/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings.csv",
                      encoding = "ISO-8859-1", low_memory=False)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
listings.head()

In [ ]:
listings.info()

In [ ]:
listings["city"].value_counts()

In [ ]:
paris_listings = listings[listings["city"] == "Paris"]
paris_listings = paris_listings.loc[:,["listing_id","host_since","neighbourhood","city","accommodates","price"]]

In [ ]:
paris_listings

In [ ]:
paris_listings.info()

In [ ]:
paris_listings.isna().sum()

In [ ]:
paris_listings.dropna(how = "any", inplace = True)

In [ ]:
paris_listings.info()

In [ ]:
paris_listings["host_since "] = pd.to_datetime( paris_listings["host_since"])

In [ ]:
paris_listings.info()

In [ ]:
paris_listings.describe()

In [ ]:
paris_listings = paris_listings[(paris_listings["accommodates"]!=0) | (paris_listings["price"]!=0)]

In [ ]:
paris_listings.info()

In [ ]:
paris_listings.describe()

In [ ]:
paris_listings = paris_listings[(paris_listings["price"]!=0)]

In [ ]:
paris_listings.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
sns.boxplot(paris_listings["price"])


In [ ]:
sns.kdeplot(paris_listings["price"])

In [ ]:
sns.histplot(paris_listings["price"])


In [ ]:
upper_limit = np.mean(paris_listings["price"]) + 3*np.std(paris_listings["price"])


In [ ]:
paris_listings = paris_listings[(paris_listings["price"]<=upper_limit)]

In [ ]:
paris_listings.describe()

In [ ]:
paris_listings.describe(include = "object")

In [ ]:
paris_listings["neighbourhood"].value_counts()

In [ ]:
paris_listings_avg_price = paris_listings.groupby("neighbourhood").agg({"price":"mean"})
paris_listings_avg_price.sort_values("price",ascending = False,inplace = True)

In [ ]:
paris_listings_avg_price

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(data = paris_listings_avg_price,
            x = "price",
            y = paris_listings_avg_price.index,color = "#00A699")
plt.title("Neighbourhoods in Paris(Premium Localities at the Top!)")
plt.xlabel("How Expensive? (Euros)")
plt.ylabel("Prominent Localities with AirBnBs")
plt.show()

In [ ]:
paris_listings_accommodates = paris_listings.groupby("accommodates").agg({"price":"mean"})
paris_listings_accommodates.sort_values("price",ascending = False,inplace = True)

In [ ]:
paris_listings_accommodates

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x = "accommodates", y = "price", data = paris_listings_accommodates, s = 100 , color = "blue")
            
# Add a trendline
sns.regplot(x=paris_listings_accommodates.index, y = "price", data = paris_listings_accommodates,
            scatter = False,color = "red", line_kws = {"linewidth":2})
            
plt.title("Average Airbnb Price vs. Number of Guests Accommodated",fontsize=14, fontweight="bold")
plt.xlabel("Number of Guests? (Accommodates)")
plt.ylabel("Average Price(€)")
plt.grid(alpha=0.3)            
plt.show()

In [ ]:
# Convert host_since to datetime
paris_listings["host_since"] = pd.to_datetime(paris_listings["host_since"], errors="coerce")

# Now extract year
paris_listings["year_of_listing"] = paris_listings["host_since"].dt.year


In [ ]:
paris_listings.head()

In [ ]:
paris_listings_final_analysis = paris_listings.groupby("year_of_listing").agg({
    "listing_id":"count","price":"mean"  
})
paris_listings_final_analysis

In [ ]:
plt.figure(figsize = (12,6))

plt.plot(paris_listings_final_analysis.index,paris_listings_final_analysis["listing_id"],
        marker = "o", linestyle = "--", linewidth = 2, color = "teal", alpha = 0.8)
plt.title("Number of New AirBnB Listings in Paris Over Time", fontsize = 16,
         fontweight = "bold", pad =15)
plt.xlabel("Year of Listings", fontsize = 13)
plt.ylabel("Number of New Listings", fontsize = 13)

plt.grid()
plt.xticks(paris_listings_final_analysis.index, rotation = 45)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(
    x="year_of_listing",
    y="price",
    data=paris_listings_final_analysis,
    marker="o",
    linewidth=2.5,
    color="teal"
)

plt.title("Average Airbnb Prices in Paris Over Time", fontsize=18, fontweight="bold", pad=15)
plt.xlabel("Year of Listing", fontsize=14)
plt.ylabel("Average Price (€)", fontsize=14)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

# Plot number of listings (left y-axis)
ax.plot(
    paris_listings_final_analysis.index, 
    paris_listings_final_analysis["listing_id"], 
    color="teal", linewidth=2.5, marker="o", label="Number of Listings"
)
ax.set_xlabel("Year of Listing", fontsize=14)
ax.set_ylabel("Number of New Listings", fontsize=14, color="teal")
ax.tick_params(axis="y", labelcolor="teal")

# Secondary y-axis for price
ax2 = ax.twinx()
ax2.plot(
    paris_listings_final_analysis.index, 
    paris_listings_final_analysis["price"], 
    color="darkorange", linewidth=2.5, marker="s", label="Average Price (€)"
)
ax2.set_ylabel("Average Price (€)", fontsize=14, color="darkorange")
ax2.tick_params(axis="y", labelcolor="darkorange")
ax2.set_ylim(0)

# Titles and grid
plt.title("Airbnb Listings vs. Average Price in Paris (2008–2021)", fontsize=18, fontweight="bold", pad=15)
ax.grid(alpha=0.3, linestyle="--")

# Highlight regulation year (2017)
ax.axvline(x=2017, color="red", linestyle="--", linewidth=2, alpha=0.8, label="Regulation (2017)")

# Shade post-2017 period
ax.axvspan(2017, paris_listings_final_analysis.index.max(), color="red", alpha=0.1)

# Add text inside shaded area
ax.text(
    2017.5, 
    ax.get_ylim()[1]*0.9,   # position text near the top of shaded area
    "Post-regulation era", 
    color="red", fontsize=13, fontweight="bold"
)

# ---- Annotations ----
# Peak in listings (2015–2016)
peak_year = paris_listings_final_analysis["listing_id"].idxmax()
peak_val = paris_listings_final_analysis["listing_id"].max()
ax.annotate(
    f"Peak Listings\n({peak_year}: {peak_val:,})",
    xy=(peak_year, peak_val),
    xytext=(peak_year-2, peak_val+2000),
    arrowprops=dict(arrowstyle="->", color="teal"),
    fontsize=12, color="teal"
)

# Lowest price point (2014)
low_year = paris_listings_final_analysis["price"].idxmin()
low_val = paris_listings_final_analysis["price"].min()
ax2.annotate(
    f"Lowest Price\n({low_year}: €{low_val:.0f})",
    xy=(low_year, low_val),
    xytext=(low_year+1, low_val-15),
    arrowprops=dict(arrowstyle="->", color="darkorange"),
    fontsize=12, color="darkorange"
)

# Price rebound post-2017
ax2.annotate(
    "Price rebound\npost-regulation",
    xy=(2018, paris_listings_final_analysis.loc[2018, "price"]),
    xytext=(2014.5, 130),
    arrowprops=dict(arrowstyle="->", color="darkorange"),
    fontsize=12, color="darkorange"
)

# Legends
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines + lines2, labels + labels2, fontsize=12, loc="upper left")

plt.tight_layout()
plt.show()